# GITHUB ACTIVITY MONITOR

Here we will compare your stats to the rest of Github Switzerland. See how you compare !

In [66]:
YOUR_USERNAME = "dtsbourg"

In [67]:
SETTINGS = {
    't_test' : True
}

In [49]:
# Connecting to DB
from utils import get_mongo_db

import pickle

import numpy as np
import pandas as pd
# Import stats lib for t-test
from scipy import stats

In [9]:
db = get_mongo_db()

Connecting to MongoDB at localhost:27017...


In [55]:
# Get user from DB
usr = db.users.find_one({ 'login' : YOUR_USERNAME })

if usr is None:
    print("[ERROR] Sorry we couldn't find you in our DB. Make you spelled your username correctly !")
else:
    print("=> Hey " + usr['name'] + " !")

=> Hey Dylan Bourgeois !


In [56]:
# Get users from DB
res = db.users.find({ 'in_ch': True, 'repositories': { '$ne': None } })

users = []

# For each user, find his repositories
for user in res:
    repos = db.repositories.find(
        { '_id': { '$in': user['repositories'] } }
    )
    
    starred = user.get('starred', [])
    following = user.get('following', [])
    followers = user.get('followers', [])
    
    users.append({
        '_id': user['_id'],
        'login': user['login'],
        'name': user['name'],
        'location': user['location'],
        'repositories_docs': list(repos),
        'followers' : len(followers),
        'starred' : len(starred),
        'following' : len(following)
    })
    
print("Our dataset includes {} users.".format(len(users)))

all_users = pd.DataFrame(users)

Our dataset includes 5976 users.


In [57]:
current_user = all_users.loc[all_users['_id'] == usr['_id']]
rest_of_users = all_users[all_users['_id'] != usr['_id']]

## Personal metrics

### Number of repos starred

Here we will compare the number of repositories you starred compared to the rest of the swiss population.

In [63]:
print("On average, swiss users of Github have starred {} repositories.".format(int(rest_of_users['starred'].mean())))

On average, swiss users of Github have starred 43 repositories.


In [65]:
print("You have starred {} repositories.".format(int(current_user['starred'])))

You have starred 577 repositories.


In [73]:
if SETTINGS['t_test'] is True:
    (t_statistic, p_value) = stats.ttest_1samp(rest_of_users['starred'], np.mean(current_user['starred']))
    if abs(t_statistic) > 6.31:
        print("The results is statistically significant ! (95% confidence level)")
    else:
        print("The result is not provable to be statistically significant, with a t-value = {}".format(t_statistic, p_value))

The results is statistically significant ! (95% confidence level)


### Number of organisations

### Number of gists 

### Number of repos


## Social metrics

### Number of followers

### Number of stars on your repos

### Most popular repo

### How many people close to you ?

In [45]:
(t_statistic, p_value) = stats.ttest_1samp(rest_of_users['star_count'], np.mean(current_user['star_count']))
t_statistic, p_value

(4.6761679799078548, 2.9871417526811725e-06)

## Follow graph